In [1]:
import transformers
import json

import datasets
squad_dataset = datasets.load_dataset('squad')
from datasets import concatenate_datasets

import pandas as pd
import json
import numpy as np
covid_file = '../data/COVID-QA.json'
jsonfile = open(covid_file, 'r')

covidQA = jsonfile.read()
jsonfile.close()

covid_data = json.loads(covidQA)
def convert_to_squad_format(data):
    covid_qa_squad_format = []
    for rows in data['data']:
      for context in rows['paragraphs']:
        for qa_pairs in context['qas']:
          features = {'id':str(context['document_id']),
                      'title': 'COVID_19',
                      'context':str(context['context']),
                      'question':qa_pairs['question'],
                      'answers':{'answer_start': np.array([qa_pairs['answers'][0]['answer_start']], dtype=np.int32),
                                 'text':[qa_pairs['answers'][0]['text']]}}
          covid_qa_squad_format.append(features)
    covid_df = pd.DataFrame(covid_qa_squad_format)
    return (datasets.Dataset.from_dict(covid_df))#.train_test_split(test_size=0.2)

def make_and_save_full_dataset(covid,squad, path):
    full_data = datasets.dataset_dict.DatasetDict({'covid':covid_qa, 'squad':squad_qa})
    full_data.save_to_disk("../data/full_squad_covidQA")

Reusing dataset squad (C:\Users\unitu\.cache\huggingface\datasets\squad\plain_text\1.0.0\4fffa6cf76083860f85fa83486ec3028e7e32c342c218ff2a620fc6b2868483a)


In [2]:
covid_qa = convert_to_squad_format(covid_data)
squad_qa = concatenate_datasets([squad_dataset['train'], squad_dataset['validation']])
covid_and_squad_dataset_path = "../data/full_squad_covidQA"

make_and_save_full_dataset(covid_qa,squad_qa,covid_and_squad_dataset_path)

In [ ]:
import run_qa
%run run_qa.py \
  --model_name_or_path roberta-base \
  --dataset_name ../data/full_squad_covidQA/ \
  --do_train \
  --do_eval \
  --do_predict \
  --per_device_train_batch_size 10\
  --per_device_eval_batch_size 10\
  --evaluation_strategy "no" \
  --save_strategy "no" \
  --logging_strategy "epoch" \
  --learning_rate 3e-5 \
  --num_train_epochs 1 \
  --max_seq_length 384 \
  --doc_stride 128 \
  --k_fold_cross_valid 5 \
  --output_dir ../models/gradual_ft_baseline \
  --overwrite_output_dir

../data/full_squad_covidQA/
../data/full_squad_covidQA/
06/20/2021 12:22:09 - WARNING - __main__ -   Process rank: -1, device: cuda:0, n_gpu: 1distributed training: False, 16-bits training: False
06/20/2021 12:22:09 - INFO - __main__ -   Training/evaluation parameters TrainingArguments(output_dir=../models/gradual_ft_baseline, overwrite_output_dir=True, do_train=True, do_eval=True, do_predict=True, evaluation_strategy=IntervalStrategy.NO, prediction_loss_only=False, per_device_train_batch_size=10, per_device_eval_batch_size=10, gradient_accumulation_steps=1, eval_accumulation_steps=None, learning_rate=3e-05, weight_decay=0.0, adam_beta1=0.9, adam_beta2=0.999, adam_epsilon=1e-08, max_grad_norm=1.0, num_train_epochs=1.0, max_steps=-1, lr_scheduler_type=SchedulerType.LINEAR, warmup_ratio=0.0, warmup_steps=0, logging_dir=runs\Jun20_12-22-09_DESKTOP-1CU6Q06, logging_strategy=IntervalStrategy.EPOCH, logging_first_step=False, logging_steps=500, save_strategy=IntervalStrategy.NO, save_steps=50

[INFO|configuration_utils.py:517] 2021-06-20 12:22:10,711 >> loading configuration file https://huggingface.co/roberta-base/resolve/main/config.json from cache at C:\Users\unitu/.cache\huggingface\transformers\733bade19e5f0ce98e6531021dd5180994bb2f7b8bd7e80c7968805834ba351e.35205c6cfc956461d8515139f0f8dd5d207a2f336c0c3a83b4bc8dca3518e37b
[INFO|configuration_utils.py:553] 2021-06-20 12:22:10,713 >> Model config RobertaConfig {
  "architectures": [
    "RobertaForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "eos_token_id": 2,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-05,
  "max_position_embeddings": 514,
  "model_type": "roberta",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 1,
  "position_embedding_type": "absolute",
  "transformers_version": "4.6.1",
  "type_vocab_size": 1,
  "u

Datasets {'validation': Dataset({
    features: ['answers', 'context', 'id', 'question', 'title'],
    num_rows: 202
}), 'test': Dataset({
    features: ['answers', 'context', 'id', 'question', 'title'],
    num_rows: 202
}), 'train': Dataset({
    features: ['id', 'title', 'context', 'question', 'answers'],
    num_rows: 99784
})}


[INFO|trainer.py:1156] 2021-06-20 12:23:26,638 >> ***** Running training *****
[INFO|trainer.py:1157] 2021-06-20 12:23:26,658 >>   Num examples = 153013
[INFO|trainer.py:1158] 2021-06-20 12:23:26,673 >>   Num Epochs = 1
[INFO|trainer.py:1159] 2021-06-20 12:23:26,688 >>   Instantaneous batch size per device = 10
[INFO|trainer.py:1160] 2021-06-20 12:23:26,689 >>   Total train batch size (w. parallel, distributed & accumulation) = 10
[INFO|trainer.py:1161] 2021-06-20 12:23:26,691 >>   Gradient Accumulation steps = 1
[INFO|trainer.py:1162] 2021-06-20 12:23:26,692 >>   Total optimization steps = 15302


Step,Training Loss


In [ ]:
# #for making the squad dataset smaller we can do split up the full dataset like so
# k_fold = 3
# temp = squad_qa.train_test_split(test_size=0.5)
# squad_qa = temp['train']
#
# make_and_save_full_dataset(covid_qa,squad_qa,covid_and_squad_dataset_path)